# Disks Temperature (T$_{mm}$) from resolved disks 

Leonardo Testi - Gothenburg 5 June 2016

We use the fits made to the resolved Lupus disks (and the masses computed with the disk models) to attempt to derive an empirical relationship for T$_{mm}$ such that applying this T$_{mm}$ to unresolved disk photometry we derive an approximately correct value of the disk mass.


In [ ]:
import numpy as np
import scipy.stats as ss
import matplotlib
import matplotlib.pyplot as plt
import astropy.io.fits as aiof
import astropy.io.ascii as aio
from spectral_cube import SpectralCube
import pyregion
import astropy.units as u
from astropy.coordinates import SkyCoord
import aplpy
import os

%matplotlib inline 
matplotlib.rcParams['axes.linewidth'] = 2.5
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.size'] = 20

